### Testing specific issues with the CV's
Some testing... Sara's failed due to the "ć"

In [27]:
import pdfplumber
with pdfplumber.open("C:\Projects\Staffing\data\CV's\Sara_Vera_Marjanovi_.pdf") as pdf:
    print(pdf.pages[0].extract_text()[:-60])
    #for j, page in enumerate(pdf.pages):
    #    print(page.extract_text())

Sara Vera Marjanović
Data Scientist
Data scientist
Sara is a driven data scientist with a particular specialty in Machine Learning, Deep learning, and Natural Language 
Processing. She has end-to-end experience with wrangling a wide variety of different data sources (from free text to 
biometric data) to create either simple (e.g. clustering, regression) and complex models (e.g. Generative Adversarial 
Models) in the production pipeline. Therefore, she is well-versed in ideation, data collection, data wrangling and 
exploration, as well as model development and trouble-shooting and is comfortable with Python (especially the libraries 
Numpy, Pandas, Tensorflow, Pytorch) as well as SQL, R, and Azure DevOps.
Within her model creation and data exploration, Sara takes special care to limit bias within the dataset and model output, 
and to ensure interpretability of the results. She is diligent and conscientious with her data storytelling and data 
visualization, making her a skilled commun

### Extracting text as string from pdf', save to .csv
Here we read the text from all CV's and save the texts in a .csv file. Each line in the .csv contains name of the colleague, the page number of the CV and the text on that specific page. 

In [47]:
import csv
import glob
import regex as re
import pdfplumber
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Get list of CV paths
paths = glob.glob("C:\Projects\Staffing\data\CV's\*.pdf")
#print(paths)

stop_words = set(stopwords.words('english'))

with open("cv_test.csv", "w", newline='') as f:
    writer = csv.writer(f)
    csv_input = ["colleague", "page_number", "page_text", "word_tokens"]
    writer.writerows([csv_input])
    for i, path in enumerate(paths):
        print(f"Currently at path: {path[31:-4]}")
        with pdfplumber.open(path) as pdf:
            for j, page in enumerate(pdf.pages):

                text = page.extract_text()
                text = re.sub('[^A-Za-z0-9 .,]+', '', text)
                word_tokens = word_tokenize(text)
                filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
                # Only special characters are removed.
                csv_input = [path[31:-4],j+1, text, filtered_sentence]
                #Write page text to pdf
                writer.writerows([csv_input])

Currently at path: Antonio_Arfe
Currently at path: Ashish_Meshram
Currently at path: Ayman_Eleya_Zaki
Currently at path: Aziz_Ari
Currently at path: Branislav_Machava
Currently at path: Daniel_Masson
Currently at path: Daria_Korzel
Currently at path: Devichand_Das
Currently at path: Dinesh_Kumar_Padala
Currently at path: Ditlev_J_rgensen
Currently at path: Drisya_Thumba
Currently at path: Emil_Djursner_Rasmussen
Currently at path: Eugenia_Borgia
Currently at path: Hamza_Minhas
Currently at path: Hatef_Abdollahi
Currently at path: Jacob_Karlstr_m
Currently at path: Jan_Neldeberg
Currently at path: Jayant_Pant
Currently at path: Jes_Melbye_Chergui
Currently at path: Julian_Eikhaug
Currently at path: Kristina_Kj_r_Hedegaard
Currently at path: Ksheeraja_Vinjanampati_
Currently at path: Lee_Ann_Zambrano_Larsen
Currently at path: Line_Bruun
Currently at path: Mohd_Arshad
Currently at path: Mohit_Kumar_Bhati
Currently at path: Mostafa_Ellabaan
Currently at path: Olivia_Essen
Currently at path

### Create TaggedDocument, train Doc2Vec model on data
First we created a TaggedDocument object where we tag each document with the respective name. Each document is a page on the CV.

In [48]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd

df = pd.read_csv("cv_test.csv")

tagged_data = []
tags = df["colleague"].unique().tolist()
#print(tags)
#cnt = 0
"""
if used on text
for tags in tags:
    docs = df[df["colleague"] == tags]["page_text"].tolist()
    for doc in docs:
        tagged_data.append(TaggedDocument(words=word_tokenize(doc), tags=[str(tags)]))
    
    #cnt += 1
"""

# if used on word tokens that are filtered
for tags in tags:
    docs = df[df["colleague"] == tags]["word_tokens"].tolist()
    for doc in docs:
        tagged_data.append(TaggedDocument(words=doc, tags=[str(tags)]))


print(tagged_data[0])
#1/0
#tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(name)]) for name, _d in (df["colleague"], df["page_text"])]
max_epochs = 20
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=5,
                dm =1)

model.build_vocab(tagged_data) # tagged_data

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged_data
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_cvs.model")
print("Model Saved")

TaggedDocument<['Antonio', 'ArfeSenior', 'Big', 'Data', 'EngineerSenior', 'Data', 'Engineer', 'Antonio', 'experienced', 'SoftwareData', 'Engineer', 'interested', 'anything', 'technically', 'challenging', '.', 'worked', 'wide', 'range', 'projects', 'Enterprise', 'level', ',', 'gathering', 'solid', 'experience', 'productionizing', 'infrastructure', 'applications', 'data', 'pipelines', 'support', 'Data', 'Scientists.He', 'worked', 'two', 'main', 'areas', 'Design', ',', 'implement', 'maintain', 'infrastructure', 'applications', 'run', 'years', ',', 'highly', 'available', 'fault', 'tolerant', 'Build', 'data', 'pipelines', 'productionize', 'advanced', 'analytics', 'models', 'constant', 'need', 'changes', 'new', 'featuresHIGHLIGHTS', 'Explain', 'kind', 'Distributed', 'software', 'system', 'technical', 'nontechnical', 'audiences', '.', 'Help', 'data', 'scientists', 'productionize', 'models', 'Catch', 'errors', 'implementation', 'starts', 'CICD', 'mechanisms', 'Teamwork', 'international', 'envi

### Use trained model to find best match between a subject and a colleague.
Here we use the trained model to match which CV best fits the given text (subject tested on is Data Migration).

In [49]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")


# Test random text on data migration
data_migration = """Data Migration
Data-Migration-Icon-603x603What is Data Migration?
Data Migration is the process of selecting and moving data from one location to another – this may involve moving data across different storage vendors, and across different formats.

Data migrations are often done in the context of retiring a system and moving to a new system, or in the context of a cloud migration, or in the context of a modernization or upgrade strategy.

Data migrations can be laborious, error prone, manual, and time consuming. Migrating data may involve finding and moving billions of files, which can succumb to storage and network slowdowns or outages. Also, different file systems do not often preserve metadata in exactly the same way, so migrating data without loss of fidelity and integrity can be a challenge.

Network Attached Storage (NAS) migration is the process of migrating from one NAS storage environment to another. This may involve migrations within a vendor’s ecosystem such as NetApp data migration to NetApp or across vendors such as NetApp data migration to Isilon or EMC to NetApp or EMC to Pure FlashBlade. A high-fidelity NAS migration solution should preserve not only the file itself but all of its associated metadata and access controls.

Network Attached Storage (NAS) to Cloud data migration is the process of moving data from an on-premises data center to a cloud. It requires data to be moved from a file format (NFS or SMB) to an Object/Cloud format such as S3. A high-fidelity NAS-to-Cloud migration solution preserves all the file metadata including access control and privileges in the cloud. This enables data to be used either as objects or as files in the cloud.

Storage migration is a general-purpose term that applies to moving data across storage arrays.

Data migrations typically involve four phases:

Planning – Deciding what data should be migrated. Planning may often involve analyzing various sources to find the right data sets. For example, several customers today are interested in upgrading some data to Flash – finding hot, active data to migrate to Flash can be a useful planning exercise.
Initial Migration – Do a first migration of all the data. This should involve migrating the files, the directories and the shares.
Iterative Migrations – Look for any changes that may have occurred during the initial migration and copy those over.
Final Cutoff – A final cutoff involves deleting data at the original storage and managing the mounts, etc., so data can be accessed from the new location going forward.
Resilient data migration refers to an approach that automatically adjusts for failures and slowdowns and retries as needed. It also checks the integrity of the data at the destination to ensure full fidelity.

Types of data migration
When it comes to file data, there are NAS Migrations and Cloud Migrations. There are also NAS migrations to the cloud. Data migrations are often seen as a dreaded and laborious part of the storage management lifecycle. Free tools are often considered first but they can introduce risk, time and cost overruns and they are typically labor intensive and error-prone. On the other hand, traditional migration tools have complex legacy architectures and are expensive point products that do not provide ongoing value – resulting in sunk costs.

Look for simple, fast, reliable data migration tools are not one-and-done point tools. The right data migration solution should be able to handle other data management use cases, including data tiering and data replication.

How to plan a Smart NAS or Cloud data migration?
The typical steps for any data migration project are:

Analytics: Before you start an unstructured data migration project, it’s important to have visibility into:

 How fast is your data growing?
 How much data is hot vs. cold cold?
 Who is using your data?
Savings: Estimate how much you’ll save by moving to the new NAS or cloud infrastructure. This information will guide which NAS or cloud storage mix is best for your data.

Offload heavy lifting: Your data migration solution should be able to manage multiple iterations of the migration and handle problems by automatically retrying in a
slowdown or a network or storage failure.

Preserve data integrity: Your data migration solution should provide MD5 checksum on every file and assure all metadata and access controls migrate to the new environment.

Avoid sunk costs: File data migrations are a lot of heaving lifting. Your data migration solution should include automatic parallelization at every level for elastic
scaling and the ability to migrate petabytes of data seamlessly and reliably.

Reduce downtime: It is recommended that your data migration solution runs multiple iterations for more efficient cutovers.

Komprise and data migration
Komprise Elastic Data Migration is included in the Komprise Intelligent Data Management platform or is available standalone. Designed for cloud migrations and NAS migrations, with Komprise Elastic Data Migration you can run, monitor, and manage hundreds of data migrations faster than ever at a fraction of the cost. Learn more about Komprise Smart Data Migrations.

Data migration and the cloud
As unstructured data continues to grow exponentially, organizations struggle to control costs for file data storage. Many are turning to the cloud to scale and manage spend. However, choosing the right files to move can be challenging as there can easily be billions of files. Many enterprises have over 1 PB of data, which represents roughly 3 billion files. This unstructured data is growing exponentially and resides in multi-vendor storage silos for access by various applications and departments.
For these reasons, organizations often lack visibility into file data and are making decisions in the dark. To be agile and competitive, IT teams must evolve storage management to become a holistic data management strategy. The right approach to data migration and the cloud for file and object data is to use analytics in cloud data management:

Understand your data patterns
Plan using a cost model
Use data to drive stakeholder buy-in
Eliminate user disruption
Create a systematic plan for ongoing data management
Read the eBook: 5 Ways to Use Analytics for Cloud Data Migrations

Top data migration challenges
Businesses today are looking at modernizing storage and moving to a multi-cloud strategy. As they evolve to faster, flash-based Network Attached Storage (NAS) and the cloud, migrating data into these environments can be challenging. The goal is to migrate large production data sets quickly, without errors, and without disruption to user productivity.

The top cloud data migration challenges are:

How do you manage cloud data migrations without downtime?
How can you automate cloud data migrations to eliminate manual effort?
How can you ensure all the permissions, ACLs, metadata are copied correctly during a cloud data migration so you can access the data in the cloud as files?
You can overcome these challenges with some planning and automation that preserves file-based access both from on-premises and the cloud.

Data migration tools
Free Tools: Require a lot of babysitting and are not reliable for migrating large volumes of data.
Point Data Migration Solutions: Have complex legacy architectures and create sunk costs.
Komprise Elastic Data Migration: Makes cloud data migrations simple, fast, reliable and eliminates sunk costs since you continue to use Komprise after the migration. Komprise is the only solution that gives you the option to cut 70%+ cloud storage costs by placing cold data in Object classes while maintaining file metadata so it can be promoted in the cloud as files when needed."""

data_migration = re.sub('[^A-Za-z0-9 .,]+', '', data_migration)


# Get vector of test text
test_vector = model.infer_vector(word_tokenize(data_migration))

# Get all CV's that match
best_matches = model.dv.similar_by_vector(test_vector)

#See all matches
print(best_matches)
#print(best_matches[0][0])
best_match = best_matches[0][0]
# Print the text of the best match
# To analyse if there match is decent or not.
match = df[df["colleague"] == best_match]["page_text"].tolist()
#print(match)


data_science = """ 
Data Science Buzzwords and What They Mean
Algorithm: A specified and defined step-by-step process used to solve logical problems. Computers carry out tasks quickly and accurately using algorithms. Algorithms are ideal for sorting, processing, and analyzing large databases (Career Foundry).
Artificial Intelligence (AI): Technology that attempts to reproduce any component of human intellect on a larger scale and possesses advanced cognitive abilities. AI does more than merely collect data and carry out a specified job. It applies a certain level of intelligence to analyze and gain insights from the data (Forbes). The most critical component of AI is data. Efficiently running your algorithms and models will require robust data sets stored in a cloud environment. Cloud computing continues to play a significant role in shaping AI use cases (Towards Data Science, Medium).
Big Data: A large amount of data that can't be stored and processed in a reasonable amount of time using typical computing methods. The three key traits of big data are volume (amount of data rendered), velocity (speed of data rendering), and variety (types of data) (The Startup - Medium).
Business Intelligence (BI): A subset of data analytics, BI transforms data into actionable insights that help a business make strategic and tactical decisions. BI tools access and analyze data to show data-driven insights about an organization's current status in dashboards, graphs, charts, reports, maps, and summaries (CIO).
Data Anonymization: By removing or encoding identifiers, data anonymization protects private or secret information in stored data. Data anonymization rules ensure that a business recognizes and upholds its responsibility to protect sensitive, personal, and confidential information (Corporate Finance Institution). Many business’ end up with Personal Identifiable Information (PII) that they need to protect from even people inside their own organizations from seeing. 
Data Catalog / Metadata (Dataversity): 
A data catalog organizes and simplifies access to all of an organization's data assets.

The description and summary used to categorize, organize, and tag data for easier sorting and searching is referred to as metadata.

Data Governance: The process of overseeing the accessibility, integrity, consumption, and security of data in business systems through company standards and regulations. Effective data governance allows businesses to meet data privacy requirements and rely on data analytics to streamline processes and make better business decisions (TechTarget).
Data Fabric / Data Mesh (Dataversity): 
A data fabric intends to form a single management layer over distributed data and takes an architectural approach to data access. 

A data mesh promotes decentralized teams to manage data their way (along with common standard governance rules) and focuses more on linking data processes and users.

Data-Informed / Data-Driven / Data-Centric (ProgrameSecure, The Data Scientist)
The lowest level of data integration, a data-informed business collects and stores data and has it readily available for employees to access. Decision-makers are aware of the data and reference it to make decisions. However, human intuition and experience supersede data when making a decision.

As the second level of data integration, a data-driven business collects and stores data and largely depends on it to make decisions. A data-driven approach requires big data management and systems/tools/algorithms to process and filter data. 

At the final stage of data integration, a data-centric business prioritizes data to the point that it takes a prominent and permanent role in the organization's processes and decisions. Data science is a vital part of the company's operations, with entire departments dedicated to various data-related duties. 

Data Mining: The practice of evaluating a huge batch of data to find patterns and trends. By relying on big data and advanced computing methods (AI and machine learning), data mining works to uncover correlations that can lead to inferences or predictions from unstructured or massive data sets (Investopedia).
Data Quality: Refers to how well a dataset satisfies the needs of a user. High data quality has sufficient amount and detail to satisfy its intended use and is a requirement to make accurate data-driven decisions (Data Quality and Record Linkage Techniques).
Data Science: A multidisciplinary approach using mathematics, statistics, artificial intelligence, and computer engineering to study data to gain useful business insights. Helps answer the what, why, and how something happened and enables a business to make informed predictions and decisions (AWS). 
Data Scientist / Data Analyst / Data Engineer: (Toward Data Science)
Data scientists leverage statistics, mathematics, programming, and big data to solve business problems. Data scientists are typically well-versed in SQL, Python, R, and cloud technology. 

Data analysts analyze data and communicate results to make business decisions. Data analysts possess excellent communication skills and are well-versed in business operations, SQL, BI tools, Python, and R. 

Data engineers build and optimize systems to allow data scientists and analysts to do their work. Data engineers possess strong programming skills and are well-versed in SQL, Python, cloud, big data, and distributed computing.  

Data Transformation: The process of modifying the format, structure, or values of data to organize it better and make it easier to use. Properly structured and verified data helps mitigate data processing errors and improves compatibility between different applications and systems (Stitch).
Data Visualization: A graphical presentation of data that makes it easier for decision-makers to quickly find and analyze trends, anomalies, and patterns in their data. Data visualization can come in the form of dashboards, charts, graphs, maps, plots, infographics, and more (CIO).
Data Warehouse / Data Lake / Data Lakehouse: (Big Data Processing, Medium)
A data warehouse serves as a centralized storage system for organized data. Optimized for processing and storing regularly collected data from multiple sources and formats, data warehouses provide better data quality and quicker query responses. A robust data warehouse with a lot of historical data can be great for ML workloads.

A data lake is a high-capacity data storage with a processing layer to store structured, semi-structured, and unstructured data. It can process and store raw data where its use is still unknown. While great for complex tasks like machine learning and predictive analytics, data lakes have poor data quality and require additional tools to run queries.

A data lakehouse features a combination of data warehouse and data lake capabilities in a single hybrid platform. Data lakehouses provide the form and structure of a data warehouse while offering unstructured data storage like a data lake, allowing data users to get information quickly and apply it right away.

Data Wrangling: Manual or automated cleaning and processing of raw data to transform it into more functional formats for use and analysis. Data wrangling methods can vary and depend on the data and project objectives (Harvard Business School Online).
Extract, Transform, Load (ETL) / Extract, Transform, Load (ELT): (AWS)
ETL transforms data before loading it into a data warehouse using Apache Spark or Apache Hive on Amazon EMR or AWS Glue. ETL allows you to pick and choose the tools you want to use for data manipulations.

ELT loads data into the data warehouse and then transforms the data within the data warehouse using SQL semantics and Massively Parallel Processing (MPP) architecture's highly optimized and scalable data storage and processing power. A data transformation engine is already built into the data warehouse.

Hadoop: Apache Hadoop is an open source framework used to efficiently store and process large-sized datasets ranging from gigabytes to petabytes of data. Hadoop uses the clustering of multiple computers to simultaneously analyze massive datasets faster than using one large computer to store and process data (AWS).
Machine Learning: Not synonymous with AI, machine learning is a subset of AI and is widely used in modern practice. ML uses sophisticated mathematical algorithms to detect patterns in large amounts of data to predict outcomes. Trained on representative data and solutions, ML is built and refined over time with increased amounts of data (Forbes). 
Natural Language Processing (NLP): A method that enables computers to intelligently analyze, understand, and derive meaning from textual data. NLP lets you extract key phrases, sentiment, syntax, key elements (brand, date, place, person, etc.), as well as the language of the text. NLP can be used to gauge customer sentiment, determine the intent and the context of articles, and personalize content for your customers (AWS).
Neural Network / Deep Learning: (WGU)
A neural network is a type of machine learning that accepts data and generates an output based on its knowledge and examples. Used by machines to adapt and learn without reprogramming, a neural network mimics a human brain in which a neuron/node addresses a small portion of a problem and passes that knowledge to other neurons/nodes until they collectively find the solution to the whole problem.

Deep learning is essentially a big neural network that's referred to as a deep neural network. With many hidden layers significantly larger than typical neural networks, deep neural networks are a subset of machine learning that uses artificial neural networks (not just algorithms) that can store and process more data than standard neural networks.

Spark: A widely used open source distributed processing engine for big data applications. Apache Spark provides general batch processing, streaming analytics, machine learning, graph databases, ad hoc queries, and leverages in-memory caching and optimized execution for rapid performance (AWS).
Structured / Unstructured Data: (G2)
Structured data is commonly classified as quantitative data, and is highly structured, formatted, and organized that fits nicely into specified fields and columns. Names, dates, addresses, credit card numbers, stock information, and geolocation are examples of structured data.

Unstructured data is commonly classified as qualitative data, and is difficult to collect, process, and analyze because it lacks organization and a set framework. Text, video files, audio files, mobile activity, social network posts, satellite images, and surveillance imaging are examples of unstructured data.

X Analytics: A term coined by Gartner, X Analytics refers to the capability to perform all types of analytics on an organization's structured and unstructured data regardless of its format and storage location. The "X" in X Analytics represents the data variable for different structured and unstructured content (The New Stack).

"""
data_science = re.sub('[^A-Za-z0-9 .,]+', '', data_science)
# Get vector of test text
test_vector = model.infer_vector(word_tokenize(data_science))

# Get all CV's that match
best_matches = model.dv.similar_by_vector(test_vector)

#See all matches
print(best_matches)
#print(best_matches[0][0])
best_match = best_matches[0][0]
# Print the text of the best match
# To analyse if there match is decent or not.
match = df[df["colleague"] == best_match]["page_text"].tolist()
#print(match)



Model loaded!
[('Branislav_Machava', 0.4412524104118347), ('Vicky_Singh_Kumar', 0.437232106924057), ('Mohd_Arshad', 0.43550431728363037), ('Verma_Kanika', 0.4134233891963959), ('Dinesh_Kumar_Padala', 0.4101724922657013), ('Aziz_Ari', 0.40073031187057495), ('Ayman_Eleya_Zaki', 0.39506959915161133), ('Shivam_Singh', 0.3085992932319641), ('S_ren_Andersen', 0.30363619327545166), ('Jes_Melbye_Chergui', 0.2986653745174408)]
[('Mohd_Arshad', 0.41555675864219666), ('Vicky_Singh_Kumar', 0.4111272990703583), ('Branislav_Machava', 0.39478161931037903), ('Dinesh_Kumar_Padala', 0.3579641282558441), ('Verma_Kanika', 0.3533032536506653), ('Ayman_Eleya_Zaki', 0.3512270450592041), ('S_ren_Andersen', 0.3496091067790985), ('Shabina_Bibi', 0.3295360505580902), ('Aziz_Ari', 0.32760512828826904), ('Shivam_Singh', 0.30753326416015625)]
